# Day 1

## part 1

In [126]:
import re


pattern = re.compile(r"\d")
calibration_sum = 0

with open("inputs/01.txt") as f:
    for line in f:
        nums = pattern.findall(line)
        calibration_sum += int(f"{nums[0]}{nums[-1]}")

calibration_sum

54927

## part 2

In [127]:
spelled_numbers = {
    "one": "1",
    "two": "2",
    "three": "3",
    "four": "4",
    "five": "5",
    "six": "6",
    "seven": "7",
    "eight": "8",
    "nine": "9",
}

# handle overlapping matches like "eighthree"
pattern = re.compile(f"(?=(\d|{'|'.join(spelled_numbers.keys())}))")
calibration_sum = 0

with open("inputs/01.txt") as f:
    for line in f:
        nums = [spelled_numbers.get(x, x) for x in pattern.findall(line)]
        calibration_sum += int(f"{nums[0]}{nums[-1]}")

calibration_sum

54581

# Day 2

## part 1

In [129]:
import re
import math


max_cubes = {
    "red": 12,
    "green": 13,
    "blue": 14
}

pattern = re.compile(r"(\d+) (red|green|blue)")
valid_ids_sum = 0

with open("inputs/02.txt") as f:
    for line in f:
        id, trials = line.split(":")
        id = int(id.split()[-1])
        
        trials = pattern.findall(trials)
        trials = [(color, int(num)) for num, color in trials]
        valid = [max_cubes[color] >= num for (color, num) in trials]
        
        if all(valid): valid_ids_sum += id
    
valid_ids_sum

2512

## part 2

In [130]:
power_set_sum = 0

with open("inputs/02.txt") as f:
    for line in f:
        trials = line.split(":")[-1]
        trials = pattern.findall(trials)
        trials = [(color, int(num)) for num, color in trials]

        max_color = {"red": 0, "green": 0, "blue": 0}
        
        for color, num in trials:
            max_color[color] = max(max_color[color], num)

        power_set_sum += math.prod(max_color.values())

power_set_sum

67335

# Day 3

## part 1

In [131]:
import string
import itertools
import math



def padding_row(line_len):
    return "".join(["."] * line_len + ["\n"])


def pad_row(row):
    return [".", *row.strip(), "."]


def pad_iterator(iterator):
    line_len = len(iterator.readline().strip())
    iterator.seek(0)

    return itertools.chain([padding_row(line_len)], iterator)


def part_number(top, middle, bottom, window_start, window_end, symbols):
    num = int("".join(middle[window_start:window_end]))
    
    top = set(top[window_start - 1 : window_end + 1])
    middle = set(middle[window_start - 1 : window_end + 1])
    bottom = set(bottom[window_start - 1 : window_end + 1])

    if len((top | middle | bottom) & symbols) > 0:
        return num
    else:
        return 0
    

symbols = set(string.punctuation) - {"."}
is_last_row = False
parts_sum = 0

with open("inputs/03.txt") as f:
    f = pad_iterator(f)
    
    top = next(f)
    middle = next(f)
    bottom = next(f)
    
    top, middle = pad_row(top), pad_row(middle)
    
    while bottom is not None:
        bottom = pad_row(bottom)
        window_start = window_end = 1
        found_number = False

        while window_end < len(middle):
            match found_number, middle[window_end].isnumeric():
                case True, False:
                    parts_sum += part_number(top, middle, bottom, window_start, window_end, symbols)
                    found_number = False
                case False, True:
                    found_number = True
                    window_start = window_end

            window_end += 1

        top = middle
        middle = bottom
        bottom = next(f, None)
        
        if bottom is None and not is_last_row:
            is_last_row = True
            bottom = padding_row(len(middle))

parts_sum

525911

## part 2

In [132]:
def get_number(row, position):
    left = right = position
    while row[left - 1].isnumeric():
        left -= 1
    while row[right + 1].isnumeric():
        right += 1
    return int("".join(row[left:right + 1]))


def check_top_or_bottom(row, position):
    nums = []
    if row[position].isnumeric():
        nums.append(get_number(row, position))
    else:
        if row[position - 1].isnumeric():
            nums.append(get_number(row, position - 1))
        if row[position + 1].isnumeric():
            nums.append(get_number(row, position  + 1))
    
    return nums


def check_left_or_right(row, position):
    nums = []
    if row[position].isnumeric():
        nums.append(get_number(row, position))
    
    return nums


is_last_row = False
gear_product = 0

with open("inputs/03.txt") as f:
    f = pad_iterator(f)
    
    top = next(f)
    middle = next(f)
    bottom = next(f)
    
    top, middle = pad_row(top), pad_row(middle)
        
    while bottom is not None:
        bottom = pad_row(bottom)
        
        for position, char in enumerate(middle):
            if char == "*":
                nums = [
                    *check_top_or_bottom(top, position),
                    *check_top_or_bottom(bottom, position),
                    *check_left_or_right(middle, position - 1),
                    *check_left_or_right(middle, position + 1)
                ]
                
                if len(nums) == 2:
                    gear_product += math.prod(nums)
        
        top = middle
        middle = bottom
        bottom = next(f, None)
        
        if bottom is None and not is_last_row:
            is_last_row = True
            bottom = padding_row(len(middle))

gear_product

75805607

# Day 4

## part 1

In [138]:
def get_plays(line):
    nums = line.split(":")[-1]
    winning, played = nums.split("|")
    winning, played = set(map(int, winning.split())), set(map(int, played.split()))
    
    return winning, played


def get_deck(filepath):
    deck = []
    with open(filepath) as f:
        for line in f:
            winning, played = get_plays(line)
            deck.append((winning, played))
            
    return deck


deck = get_deck("inputs/04.txt")
points = 0
for winning, played in deck:
    num_matches = len(winning & played)
    if num_matches > 0: points += 2**(num_matches - 1)

points

23847

## part 2

In [137]:
weights = [1]*len(deck)
for i, (winning, played) in enumerate(deck):
    num_matches = len(winning & played)
    
    if num_matches > 0:
        for j in range(1, num_matches + 1):
            weights[i + j] += weights[i]

sum(weights)

8570000

# Day 5

## part 1

In [121]:
import re


def parse_input(filepath):
    PARSING_INDICES = {
        "seed-to-soil map:": 0,
        "soil-to-fertilizer map:": 1,
        "fertilizer-to-water map:": 2,
        "water-to-light map:": 3,
        "light-to-temperature map:": 4,
        "temperature-to-humidity map:": 5,
        "humidity-to-location map:": 6
    }
    
    pattern = re.compile(r"\d+")
    mappings = [list() for _ in range(7)]
    current_parse_index = None
    
    with open(filepath) as f:
        for line in f:
            if len(line.strip()) == 0:
                continue
            
            if line.strip() in PARSING_INDICES:
                current_parse_index = PARSING_INDICES[line.strip()]
                continue
            
            if current_parse_index is not None:
                destination, source, size = list(map(int, pattern.findall(line)))
                mappings[current_parse_index].append((source, source + size, destination))
            else:
                seeds = list(map(int, pattern.findall(line)))
    
    for m in mappings:
        m.sort(key = lambda x: x[0])
       
    return seeds, mappings


# assumes arr is sorted (ascending)
def search(arr, val):
    left, right = 0, len(arr) - 1
    result = None
    
    while left <= right:
        mid = (left + right) // 2
        if arr[mid][0] <= val:
            result = arr[mid]
            left = mid + 1
        else:
            right = mid - 1
    
    return result


def get_location(seed, mappings):
    loc = seed
    for m in mappings:
        range_map = search(m, loc)
        if range_map is not None and loc <= range_map[1]:
            loc += range_map[2] - range_map[0]
    
    return loc


def get_locations(seeds, mappings):
    locations = [*seeds]
    for i, seed in enumerate(locations):
        locations[i] = get_location(seed, mappings)
        
    return locations


# map is encoded as (source start, source end, destination start)
seeds, mappings = parse_input("inputs/05.txt")
ans = min(get_location(s, mappings) for s in seeds)

print(ans)

424490994


## part 2

In [122]:
# seed intervals are encoded as (start, end)
# for each seed interval, we need to propagate its through the maps

# for each I_source, offset in maps:
#    for each seed interval I_seed:
#       if I_seed is outside I_source: I_seed is propagated as is
#       if I_seed is inside I_source: I_seed is shifted by offset
#       if it overlaps we need to split the interval in 2
#           part of I_seed within I_source will be shifted by offset
#           part of I_seed outside I_source is propagated as is

# we end up with a bunch of intervals

# map : (source start, source end, destination start)
# seed: (seed start, seed end)

seed_intervals = [(start, start + size - 1) for start, size in zip(seeds[::2], seeds[1::2])]
current_intervals = [*seed_intervals]

for m in mappings:
    split_intervals = []
        
    for source_start, source_end, dest_start in m:
        offset = dest_start - source_start
        tmp_intervals = []
                
        while current_intervals:
            (seed_start, seed_end) = current_intervals.pop()
            
            left = (seed_start, min(seed_end, source_start))
            mid = (max(seed_start, source_start), min(source_end, seed_end))
            right = (max(source_end, seed_start), seed_end)
            
            if left[1] > left[0]:
                # identity
                tmp_intervals.append(left)
            if mid[1] > mid[0]:
                # split interval
                split_intervals.append((mid[0] + offset, mid[1] + offset))
            if right[1] > right[0]:
                # identity
                tmp_intervals.append(right)
        
        current_intervals = tmp_intervals
    
    current_intervals = [*current_intervals, *split_intervals]
    

min(start for start, _ in current_intervals)

15290096

# Day 6

## part 1

In [139]:
import math
import re


def parse_input(filepath):
    pattern = re.compile(r"\d+")
    with open(filepath) as f:
        lines = f.readlines()

    times = list(map(int, pattern.findall(lines[0])))
    distances = list(map(int, pattern.findall(lines[1])))
    
    return times, distances


def solve_real_roots(a, b, c):
    # assumes discriminant > 0
    discriminant = b**2 - 4*a*c
    return (-b + math.sqrt(discriminant)) / (2*a),  (-b - math.sqrt(discriminant)) / (2*a)


def get_combinations(times, distances):
    result = []
    for t_max, d_record in zip(times, distances):
        solutions = solve_real_roots(-1, t_max, -d_record)
        solutions = (math.floor(solutions[0]) + 1, math.ceil(solutions[1]) - 1)
        result.append(solutions[1] - solutions[0] + 1)
        
    return result


times, distances = parse_input("inputs/06.txt")
result = get_combinations(times, distances)

math.prod(result)

1108800

## part 2

In [140]:
times, distances = parse_input("inputs/06.txt")
times = [int("".join(list(map(str, times))))]
distances = [int("".join(list(map(str, distances))))]

result = get_combinations(times, distances)

result[0]

36919753

# Day 7

## part 1

In [53]:
from collections import Counter


def get_hands(filename):
    hands = []
    with open(filename) as f:
        for line in f:
            hand, bid = line.strip().split()
            hands.append((list(hand), int(bid)))
    
    return hands


def score_hand_type(hand):
    freqs = list(Counter(hand).values())
    match (len(freqs), max(freqs)):
        case (5, _): return 0
        case (4, _): return 1
        case (3, 2): return 2
        case (3, _): return 3
        case (2, 3): return 4
        case (2, _): return 5
        case (1, _): return 6
        
    
def score_cards(hand, cards_strength):    
    # hand is encoded in base 13, convert it to base 10
    result = 0
    base = 13
    for i, card in enumerate(reversed(hand)):
        result += cards_strength[card] * (base ** i)
    
    return result


def utility(hand, cards_strength):
    return score_hand_type(hand) * 1_000_000 + score_cards(hand, cards_strength)


hands = get_hands("inputs/07.txt")
cards_strength = {c: i for i, c in enumerate('23456789TJQKA')}
utilities = [utility(hand, cards_strength) for hand, _ in hands]
ranks = [sorted(utilities).index(x) + 1 for x in utilities]
winnings = sum(rank * bid for rank, (_, bid) in zip(ranks, hands))

winnings

246409899

## part 2

In [55]:
def score_hand_type_with_jokers(hand):
    replaced_hand = [*hand]
    joker_indices = [i for i, c in enumerate(hand) if c == "J"]
    
    if len(joker_indices) > 0:
        hand_wo_jokers = [c for c in hand if c != "J"]
        
        if len(hand_wo_jokers) > 0:
            most_common = Counter(hand_wo_jokers).most_common(1)[0][0]
            for idx in joker_indices:
                replaced_hand[idx] = most_common
        else:
            replaced_hand = ["A"] * len(hand)
    
    return score_hand_type(replaced_hand)


def utility_part2(hand, cards_strength):
    return score_hand_type_with_jokers(hand) * 1_000_000 + score_cards(hand, cards_strength)


cards_strength = {c: i for i, c in enumerate('J23456789TQKA')}

utilities = [utility_part2(hand, cards_strength) for hand, _ in hands]
ranks = [sorted(utilities).index(x) + 1 for x in utilities]
winnings = sum(rank * bid for rank, (_, bid) in zip(ranks, hands))

winnings

244848487